![MuJoCo banner](https://raw.githubusercontent.com/google-deepmind/mujoco/main/banner.png)

# <h1><center>LQRチュートリアル  <a href="https://colab.research.google.com/github/google-deepmind/mujoco/blob/main/python/LQR.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

このノートブックでは、 [**MuJoCo** physics](https://github.com/google-deepmind/mujoco#readme) を使用したLQRコントローラーの例を提供します。

<!-- Copyright 2021 DeepMind Technologies Limited

     Licensed under the Apache License, Version 2.0 (the "License");
     you may not use this file except in compliance with the License.
     You may obtain a copy of the License at

         http://www.apache.org/licenses/LICENSE-2.0

     Unless required by applicable law or agreed to in writing, software
     distributed under the License is distributed on an "AS IS" BASIS,
     WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
     See the License for the specific language governing permissions and
     limitations under the License.
-->

## すべてのインポート

In [0]:
!pip install mujoco

# GPUレンダリングを設定
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# glvndがNvidia EGLドライバーを認識できるようにICD設定を追加
# これは通常Nvidiaドライバーパッケージの一部としてインストールされますが、
# ColabカーネルはAPT経由でドライバーをインストールしないため、ICDが欠落しています。
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# MuJoCoがEGLレンダリングバックエンドを使用するように設定（GPUが必要）
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

# インストールが成功したか確認
try:
  print('Checking that the installation succeeded:')
  import mujoco
  mujoco.MjModel.from_xml_string('<mujoco/>')
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')

print('Installation successful.')

# その他のインポートとヘルパー関数
import numpy as np
from typing import Callable, Optional, Union, List
import scipy.linalg

# グラフィックスとプロット
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt

# numpyの出力を見やすくする
np.set_printoptions(precision=3, suppress=True, linewidth=100)

from IPython.display import clear_output
clear_output()


## 標準的なヒューマノイドを読み込んでレンダリング

In [0]:
print('Getting MuJoCo humanoid XML description from GitHub:')
!git clone https://github.com/google-deepmind/mujoco
with open('mujoco/model/humanoid/humanoid.xml', 'r') as f:
  xml = f.read()

XMLは `MjModel` のインスタンス化に使用されます。モデルがあれば、シミュレーション状態を保持する `MjData` と、上で定義した `Renderer` クラスのインスタンスを作成できます。

In [0]:
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
renderer = mujoco.Renderer(model)

`data` オブジェクトの状態はデフォルト設定になっています。前方動力学を呼び出して、すべての導出量（ワールド座標系でのジオメトリの位置など）を計算し、シーンを更新してレンダリングしましょう：

In [0]:
mujoco.mj_forward(model, data)
renderer.update_scene(data)
media.show_image(renderer.render())

モデルにはいくつかの組み込みの「キーフレーム」が付属しており、これらは保存されたシミュレーション状態です。

`mj_resetDataKeyframe` を使用してそれらをロードできます。それらがどのように見えるか見てみましょう：

In [0]:
for key in range(model.nkey):
  mujoco.mj_resetDataKeyframe(model, data, key)
  mujoco.mj_forward(model, data)
  renderer.update_scene(data)
  media.show_image(renderer.render())

それでは物理シミュレーションを実行してビデオを作成しましょう。

In [0]:
DURATION  = 3   # 秒
FRAMERATE = 60  # Hz

# 片足立ちポーズに初期化
mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
  # シミュレーションを進める
  mujoco.mj_step(model, data)

  # レンダリングしてフレームを保存
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

# ビデオを表示
media.show_video(frames, fps=FRAMERATE)

モデルには組み込みのトルクアクチュエーターが定義されており、 `data.ctrl` ベクトルを設定することでヒューマノイドの関節を駆動できます。ノイズを注入した場合にどうなるか見てみましょう。

ここで、ヒューマノイドの重心を追跡するカスタムカメラを使用しましょう。

In [0]:
DURATION  = 3   # 秒
FRAMERATE = 60  # Hz

# 新しいカメラを作成し、より近い距離に移動
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2

mujoco.mj_resetDataKeyframe(model, data, 1)

frames = []
while data.time < DURATION:
  # 制御ベクトルを設定
  data.ctrl = np.random.randn(model.nu)

  # シミュレーションを進める
  mujoco.mj_step(model, data)

  # レンダリングしてフレームを保存
  if len(frames) < data.time * FRAMERATE:
    # 注視点をヒューマノイドの重心に設定
    camera.lookat = data.body('torso').subtree_com

    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

## 片足での安定した立位

明らかにこの初期ポーズは安定していません。 [Linear Quadratic Regulator](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator) を使用して安定化制御則を見つけてみましょう。

### LQR理論の要約

この理論を説明するオンラインリソースは多数ありますが（1960年代にRudolph Kalmanによって開発されました）、ここでは最小限の要約を提供します。

状態 $x$ と制御 $u$ に対して線形である動的システムが与えられたとき、
$$
x_{t+h} = A x_t + B u_t
$$
システムが可制御性条件を満たす場合、次のように最適な方法で安定化（$x$ を0に駆動）することが可能です。2つの対称正定値行列 $Q$ と $R$ を使用して、状態と制御に対する二次コスト関数 $J(x,u)$ を定義します：
$$
J(x,u) = x^T Q x + u^T R u
$$

価値関数としても知られるコストツーゴー $V^\pi(x_0)$ は、状態が $x_0$ から始まり、制御則 $u=\pi(x)$ を使用しながら動力学に従って進化する場合の、将来のコストの総和です：
$$
V^\pi(x_0) = \sum_{t=0}^\infty J(x_t, \pi(x_t))
$$

ここでKalmanの中心的な結果を述べることができます。コストツーゴーを最小化する最適な制御則（すべての可能な制御則の中で！）は線形です：
$$
\pi^*(x) = \underset{\pi}{\text{argmin}}\; V^\pi(x)=-Kx
$$
そして最適なコストツーゴーは二次です：
$$
V^*(x) =\underset{\pi}{\min}\; V^\pi(x) = x^T P x
$$

行列 $P$ はRiccati方程式に従います：
$$
P = Q + A^T P A - A^T P B (R+B^T P B)^{-1} B^T P A
$$
そして制御ゲイン行列 $K$ との関係は：
$$
K = (R + B^T  P B)^{-1} B^T P A
$$

### 線形化の設定点を理解する

もちろん、ヒューマノイドシミュレーションは線形とは程遠いものです。しかし、MuJoCoの `mj_step` 関数がある非線形動力学 $x_{t+h} = f(x_t,u_t)$ を計算する一方で、任意の状態-制御ペアの周りでこの関数を*線形化*できます。次の状態 $y=x_{t+h}$、現在の状態 $x=x_t$、現在の制御 $u=u_t$ のショートカットを使用し、 $\delta$ を「小さな変化」の意味で使用すると、次のように書けます：
$$
\delta y = \frac{\partial f}{\partial x}\delta x+ \frac{\partial f}{\partial u}\delta u
$$

言い換えれば、偏微分行列は $x$ と $u$ への摂動と $y$ への変化の間の線形関係を記述します。上記の理論と比較すると、線形化された動的システムを考える場合、偏微分（ヤコビアン）行列を遷移行列 $A$ と $B$ と同一視できます：
$$
A = \frac{\partial f}{\partial x} \quad
B = \frac{\partial f}{\partial u}
$$

線形化を実行するには、線形化する設定点 $x$ と $u$ を選択する必要があります。 $x$ はすでにわかっています。これは片足立ちの初期ポーズです。しかし、 $u$ についてはどうでしょうか？線形化する「最適な」制御をどのように見つけますか？

答えは逆動力学です。

### 逆動力学を使用した制御設定点の発見

MuJoCoの前方動力学関数 `mj_forward` は、導出量を伝播するために上で使用しましたが、状態とシステム内のすべての力（その一部はアクチュエーターによって生成されます）が与えられたときに加速度を計算します。

逆動力学関数は加速度を*入力*として受け取り、その加速度を生成するために必要な力を計算します。MuJoCoの [高速逆動力学](https://doi.org/10.1109/ICRA.2014.6907751) は独自に、接触を含むすべての制約を考慮に入れます。どのように機能するか見てみましょう。

望ましい位置設定点で前方動力学を呼び出し、 `data.qacc` の加速度を0に設定し、逆動力学を呼び出します：

In [0]:
mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)
data.qacc = 0  # 加速度がないことを主張
mujoco.mj_inverse(model, data)
print(data.qfrc_inverse)

逆動力学によって見つかった力を調べると、かなり不穏なものが見られます。3番目の自由度（DoF）、つまりルート関節の垂直運動DoFに非常に大きな力が適用されています。

これは、加速度がゼロであるという主張を説明するために、逆動力学がルート関節に直接適用される「魔法の」力を発明しなければならないことを意味します。ヒューマノイドを1mmだけ上下に移動させ、1$\mu$mの増分で動かした場合に、この力がどのように変化するか見てみましょう：

In [0]:
height_offsets = np.linspace(-0.001, 0.001, 2001)
vertical_forces = []
for offset in height_offsets:
  mujoco.mj_resetDataKeyframe(model, data, 1)
  mujoco.mj_forward(model, data)
  data.qacc = 0
  # 高さを `offset` だけオフセット
  data.qpos[2] += offset
  mujoco.mj_inverse(model, data)
  vertical_forces.append(data.qfrc_inverse[2])

# 垂直力が最小となる高さオフセットを見つける
idx = np.argmin(np.abs(vertical_forces))
best_offset = height_offsets[idx]

# 関係をプロット
plt.figure(figsize=(10, 6))
plt.plot(height_offsets * 1000, vertical_forces, linewidth=3)
# 最小垂直力に対応するオフセットの赤い垂直線
plt.axvline(x=best_offset*1000, color='red', linestyle='--')
# ヒューマノイドの体重の緑の水平線
weight = model.body_subtreemass[1]*np.linalg.norm(model.opt.gravity)
plt.axhline(y=weight, color='green', linestyle='--')
plt.xlabel('Height offset (mm)')
plt.ylabel('Vertical force (N)')
plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
plt.minorticks_on()
plt.title(f'Smallest vertical force '
          f'found at offset {best_offset*1000:.4f}mm.')
plt.show()

上のプロットでは、足の接触による強い非線形関係が見られます。左側では、ヒューマノイドを床に押し込むと、床から跳ね出さない事実を説明する唯一の方法は、それを**下向き**に押す大きな外力です。右側では、ヒューマノイドを床から遠ざけるにつれて、ゼロ加速度を説明する唯一の方法はそれを**上向き**に保持する力であり、足が地面に触れなくなる高さが明確にわかり、必要な力はヒューマノイドの体重（緑の線）と正確に等しく、さらに上に移動しても一定のままです。

-0.5mm付近が完璧な高さオフセット（赤い線）であり、ここではゼロ垂直加速度が内部関節力によって完全に説明でき、「魔法の」外力に頼る必要がありません。初期ポーズの高さを修正し、 `qpos0` に保存し、逆動力学の力を再度計算しましょう：

In [0]:
mujoco.mj_resetDataKeyframe(model, data, 1)
mujoco.mj_forward(model, data)
data.qacc = 0
data.qpos[2] += best_offset
qpos0 = data.qpos.copy()  # 位置設定点を保存
mujoco.mj_inverse(model, data)
qfrc0 = data.qfrc_inverse.copy()
print('desired forces:', qfrc0)

はるかに良くなりました。ルート関節への力は小さくなっています。アクチュエーターによって合理的に生成できる力が得られたので、それらを生成するアクチュエーター値をどのように見つけますか？ヒューマノイドのような単純な `motor` アクチュエーターの場合、アクチュエーションモーメントアーム行列で単純に「除算」、つまりその擬似逆行列を掛けることができます：

In [0]:
actuator_moment = np.zeros((model.nu, model.nv))
mujoco.mju_sparse2dense(
    actuator_moment,
    data.actuator_moment.reshape(-1),
    data.moment_rownnz,
    data.moment_rowadr,
    data.moment_colind.reshape(-1),
)
ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
ctrl0 = ctrl0.flatten()  # 制御設定点を保存
print('control setpoint:', ctrl0)

より精巧なアクチュエーターでは、 $\frac{\partial \texttt{ qfrc_actuator}}{\partial \texttt{ ctrl}}$ を回復するために異なる方法が必要になり、有限差分は常に簡単なオプションです。

これらの制御を前方動力学で適用し、それらが生成する力を上で印刷した望ましい力と比較しましょう：

In [0]:
data.ctrl = ctrl0
mujoco.mj_forward(model, data)
print('actuator forces:', data.qfrc_actuator)

ヒューマノイドは（ルート関節を除いて）完全にアクチュエーションされており、必要な力はすべてアクチュエーター制限内にあるため、すべての内部関節で望ましい力と完全に一致することがわかります。ルート関節にはまだわずかな不一致がありますが、小さいです。これらの制御を適用したときのシミュレーションがどのように見えるか見てみましょう：

In [0]:
DURATION  = 3   # 秒
FRAMERATE = 60  # Hz

# 状態と制御を設定点に設定
mujoco.mj_resetData(model, data)
data.qpos = qpos0
data.ctrl = ctrl0

frames = []
while data.time < DURATION:
  # シミュレーションを進める
  mujoco.mj_step(model, data)

  # レンダリングしてフレームを保存
  if len(frames) < data.time * FRAMERATE:
    # 注視点をヒューマノイドの重心に設定
    camera.lookat = data.body('torso').subtree_com
    renderer.update_scene(data, camera)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

上で作成した完全に受動的なビデオと比較すると、これははるかに優れた制御設定点であることがわかります。ヒューマノイドはまだ倒れますが、安定化しようとし、短時間成功します。

### $Q$ と $R$ 行列の選択

LQRフィードバック制御則を得るには、 $Q$ と $R$ 行列を設計する必要があります。線形構造により、解は両方の行列のスケーリングに対して不変であるため、一般性を失うことなく $R$ を単位行列として選択できます：

In [0]:
nu = model.nu  # アクチュエーター数のエイリアス
R = np.eye(nu)

$Q$ の選択はより精巧です。2つの項の和として構築します。

まず、重心（CoM）を足の上に保つバランスコストです。それを説明するために、関節空間とグローバル直交座標位置の間をマッピングする運動学的ヤコビアンを使用します。MuJoCoはこれらを解析的に計算します。

In [0]:
nv = model.nv  # DoF数のショートカット

# ルートボディ（torso）のCoMのヤコビアンを取得
mujoco.mj_resetData(model, data)
data.qpos = qpos0
mujoco.mj_forward(model, data)
jac_com = np.zeros((3, nv))
mujoco.mj_jacSubtreeCom(model, data, jac_com, model.body('torso').id)

# 左足のヤコビアンを取得
jac_foot = np.zeros((3, nv))
mujoco.mj_jacBodyCom(model, data, jac_foot, None, model.body('foot_left').id)

jac_diff = jac_com - jac_foot
Qbalance = jac_diff.T @ jac_diff

次に、関節が初期設定から離れることへのコストです。異なる関節セットに対して異なる係数が必要です：
- フリー関節は係数0になります。これはCoMコスト項によってすでに考慮されているためです。
- 左脚でのバランスに必要な関節、つまり左脚の関節と水平腹筋関節は、初期値にかなり近く留まる必要があります。
- 他のすべての関節は、より小さい係数を持つべきです。たとえば、ヒューマノイドがバランスを取るために腕を振ることができるようにです。

これらすべての関節セットのインデックスを取得しましょう。

In [0]:
# すべての関節名を取得
joint_names = [model.joint(i).name for i in range(model.njnt)]

# 関連する関節セットへのインデックスを取得
root_dofs = range(6)
body_dofs = range(6, nv)
abdomen_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
    if 'abdomen' in name
    and not 'z' in name
]
left_leg_dofs = [
    model.joint(name).dofadr[0]
    for name in joint_names
    if 'left' in name
    and ('hip' in name or 'knee' in name or 'ankle' in name)
    and not 'z' in name
]
balance_dofs = abdomen_dofs + left_leg_dofs
other_dofs = np.setdiff1d(body_dofs, balance_dofs)

Q行列を構築する準備が整いました。バランス項の係数がかなり高いことに注意してください。これには3つの別々の理由があります：
- これは最も重視することです。バランスとは、CoMを足の上に保つことを意味します。
- CoMに対する制御権限が少ない（体の関節に対して相対的に）。
- バランスの文脈では、長さの単位が「より大きい」。膝が0.1ラジアン（≈6°）曲がった場合、おそらくまだ回復できます。CoM位置が足の位置から横に10cmずれている場合、おそらく床に向かっています。

In [0]:
# コスト係数
BALANCE_COST        = 1000  # バランス
BALANCE_JOINT_COST  = 3     # バランスに必要な関節
OTHER_JOINT_COST    = .3    # その他の関節

# Qjoint行列を構築
Qjoint = np.eye(nv)
Qjoint[root_dofs, root_dofs] *= 0  # フリー関節を直接ペナルティしない
Qjoint[balance_dofs, balance_dofs] *= BALANCE_JOINT_COST
Qjoint[other_dofs, other_dofs] *= OTHER_JOINT_COST

# 位置DoFのQ行列を構築
Qpos = BALANCE_COST * Qbalance + Qjoint

# 速度への明示的なペナルティはなし
Q = np.block([[Qpos, np.zeros((nv, nv))],
              [np.zeros((nv, 2*nv))]])

### LQRゲイン行列 $K$ の計算

LQRコントローラーを解く前に、 $A$ と $B$ 行列が必要です。これらはMuJoCoの `mjd_transitionFD` 関数によって計算され、効率的な有限差分導関数を使用して計算し、設定可能な計算パイプラインを活用して変化していない量の再計算を回避します。

In [0]:
# 初期状態と制御を設定
mujoco.mj_resetData(model, data)
data.ctrl = ctrl0
data.qpos = qpos0

# AとB行列を割り当て、計算
A = np.zeros((2*nv, 2*nv))
B = np.zeros((2*nv, nu))
epsilon = 1e-6
flg_centered = True
mujoco.mjd_transitionFD(model, data, epsilon, flg_centered, A, B, None, None)

安定化コントローラーを解く準備が整いました。 `scipy` の `solve_discrete_are` を使用してRiccati方程式を解き、要約で説明した式を使用してフィードバックゲイン行列を取得します。

In [0]:
# 離散Riccati方程式を解く
P = scipy.linalg.solve_discrete_are(A, B, Q, R)

# フィードバックゲイン行列Kを計算
K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

### 安定した立位

これで安定化コントローラーを試すことができます。

ゲイン行列 $K$ を適用するには、 `mj_differentiatePos` を使用する必要があることに注意してください。これは2つの位置の差を計算します。これは重要です。なぜなら、ルートの向きは長さ4の四元数によって与えられますが、2つの四元数の差（接空間内）は長さ3だからです。MuJoCo表記では、位置（ `qpos` ）のサイズは `nq` ですが、位置の差（および速度）のサイズは `nv` です。

In [0]:
# パラメータ
DURATION = 5          # 秒
FRAMERATE = 60        # Hz

# データをリセット、初期ポーズを設定
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# 位置差dqを割り当て
dq = np.zeros(model.nv)

frames = []
while data.time < DURATION:
  # 状態差dxを取得
  mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
  dx = np.hstack((dq, data.qvel)).T

  # LQR制御則
  data.ctrl = ctrl0 - K @ dx

  # シミュレーションを進める
  mujoco.mj_step(model, data)

  # レンダリングしてフレームを保存
  if len(frames) < data.time * FRAMERATE:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)

### 最終ビデオ

上のビデオは少し期待外れです。ヒューマノイドは基本的に動きがありません。それを修正し、フィナーレのためにいくつかの装飾を追加しましょう：
- LQRコントローラーの上にスムーズなノイズを注入して、バランス動作がより顕著でありながらぎくしゃくしないようにします。
- シーンに接触力の可視化を追加します。
- ヒューマノイドの周りをカメラをスムーズに周回させます。
- より高い解像度で新しいレンダラーをインスタンス化します。

In [0]:
# パラメータ
DURATION = 12         # 秒
FRAMERATE = 60        # Hz
TOTAL_ROTATION = 15   # 度
CTRL_RATE = 0.8       # 秒
BALANCE_STD = 0.01    # アクチュエーター単位
OTHER_STD = 0.08      # アクチュエーター単位

# 新しいカメラを作成、距離を設定
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 2.3

# 接触力の可視化を有効化
scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True

# 可視化される接触力のスケールを1cm/Nに設定
model.vis.map.force = 0.01

# スムーズな周回関数を定義
def unit_smooth(normalised_time: float) -> float:
  return 1 - np.cos(normalised_time*2*np.pi)
def azimuth(time: float) -> float:
  return 100 + unit_smooth(data.time/DURATION) * TOTAL_ROTATION

# ノイズを事前計算
np.random.seed(1)
nsteps = int(np.ceil(DURATION/model.opt.timestep))
perturb = np.random.randn(nsteps, nu)

# 「バランス」と「その他」で異なるSTDを持つスケーリングベクトル
CTRL_STD = np.empty(nu)
for i in range(nu):
  joint = model.actuator(i).trnid[0]
  dof = model.joint(joint).dofadr[0]
  CTRL_STD[i] = BALANCE_STD if dof in balance_dofs else OTHER_STD

# ノイズを平滑化
width = int(nsteps * CTRL_RATE/DURATION)
kernel = np.exp(-0.5*np.linspace(-3, 3, width)**2)
kernel /= np.linalg.norm(kernel)
for i in range(nu):
  perturb[:, i] = np.convolve(perturb[:, i], kernel, mode='same')

# データをリセット、初期ポーズを設定
mujoco.mj_resetData(model, data)
data.qpos = qpos0

# より高い解像度で新しいレンダラーインスタンス
renderer = mujoco.Renderer(model, width=1280, height=720)

frames = []
step = 0
while data.time < DURATION:
  # 状態差dxを取得
  mujoco.mj_differentiatePos(model, dq, 1, qpos0, data.qpos)
  dx = np.hstack((dq, data.qvel)).T

  # LQR制御則
  data.ctrl = ctrl0 - K @ dx

  # 摂動を追加、ステップをインクリメント
  data.ctrl += CTRL_STD * perturb[step]
  step += 1

  # シミュレーションを進める
  mujoco.mj_step(model, data)

  # レンダリングしてフレームを保存
  if len(frames) < data.time * FRAMERATE:
    camera.azimuth = azimuth(data.time)
    renderer.update_scene(data, camera, scene_option)
    pixels = renderer.render()
    frames.append(pixels)

media.show_video(frames, fps=FRAMERATE)